# Loading Raw Ephys Data (AP and LFP band)

In [ ]:
# Turn off logging, this is a hidden cell on docs page
import logging
logger = logging.getLogger('ibllib')
logger.setLevel(logging.CRITICAL)

Raw electrophysiology data recorded using spikeglx and compressed using mtscomp

## Relevant datasets
* \_spikeglx_ephysData*.cbin
* \_spikeglx_ephysData*.meta
* \_spikeglx_ephysData*.ch

## Loading

### Option 1: Stream snippets of raw ephys data

In [ ]:
from one.api import ONE
from brainbox.io.spikeglx import stream

one = ONE()

pid = 'da8dfec1-d265-44e8-84ce-6ae9c109b8bd'

time = 100 # timepoint in recording to stream
time_win = 1 # number of seconds to stream
band = 'ap' # either 'ap' or 'lf'

sr, t0 = stream(pid, time, nsecs=time_win, typ=band, one=one)

### Option 2: Download all of raw ephys data

<div class="alert alert-warning">
Warning.

The raw ephys data is very large and downloading will take a long period of time
</div>

In [ ]:
from one.api import ONE
from ibllib.io import spikeglx
one = ONE()

pid = 'da8dfec1-d265-44e8-84ce-6ae9c109b8bd'
eid, probe = one.pid2eid(pid)

band = 'ap' # either 'ap','lf'

# Find the relevant datasets and download
dsets = one.list_datasets(eid, collection=f'raw_ephys_data/{probe}', filename=f'*{band}*', details=True)
dset_paths = one._download_datasets(dsets)

# Find the .bin file
bin_file = next(dset_paths[0].parent.glob(f'*{band}.*bin'))

# Use spikeglx reader to read in the raw data
sr = spikeglx.Reader(ap_bin_file)

## More details
* [Details of raw ap datasets](https://docs.google.com/document/d/1OqIqqakPakHXRAwceYLwFY9gOrm8_P62XIfCTnHwstg/edit#heading=h.ms0y69xbzova)
* [Details of raw lfp datasets](https://docs.google.com/document/d/1OqIqqakPakHXRAwceYLwFY9gOrm8_P62XIfCTnHwstg/edit#heading=h.nct1c3j9tedk)
* [Details of mtscomp compression algorithm](https://github.com/int-brain-lab/mtscomp#readme)
* Spikesorting white paper (COMING SOON)

## Useful modules
* [ibllib.io.spikeglx](https://int-brain-lab.github.io/ibl-neuropixel/_autosummary/spikeglx.html)
* [ibllib.voltage.dsp](https://int-brain-lab.github.io/ibl-neuropixel/_autosummary/neurodsp.voltage.html)
* [brainbox.io.spikeglx.stream](https://int-brain-lab.github.io/iblenv/_autosummary/brainbox.io.spikeglx.html#brainbox.io.spikeglx.stream)

## Exploring raw ephys data

### Example 1: Destripe AP data

In [ ]:
from neurodsp.voltage import destripe

# remove sync channel from raw data
ap_data = sr[:, :-sr.nsync].T

# apply destriping algorithm to data
ap_destriped = destripe(ap_data, fs=sr.fs)

### Example 2: Stream LFP data around task event

In [ ]:
eid, probe = one.pid2eid(pid)
stimOn_times = one.load_object(eid, 'trials', collection='alf')['stimOn_times']
event_no = 100

# Get the 1s of LFP data around time point of interest
sr, t0 = stream(pid, stimOn_times[event_no], nsecs=1, typ='lf', one=one)
lf_data = sr[:, :-sr.nsync].T

## Other relevant examples
* COMING SOON